# Import libraries

In [108]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import PIL.Image
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import skimage.io as io  
from skimage import transform
from sklearn.preprocessing import OrdinalEncoder
from tensorflow.keras.utils import to_categorical

# Loading data

In [109]:
training = pd.read_csv(".//Training_set.csv")
testing = pd.read_csv(".//Testing_set.csv")

In [110]:
training.head()

,filename,label
0,Image_1.jpg,sitting
1,Image_2.jpg,using_laptop
2,Image_3.jpg,hugging
3,Image_4.jpg,sleeping
4,Image_5.jpg,using_laptop


In [111]:
labels = training["label"]
filename = training["filename"].to_numpy()

In [112]:
train = np.zeros(shape=(len(filename),160,240,3))

In [113]:

for i in range(100):#(len(filename)):
    path = os.path.join("./train/",filename[i])
    image = io.imread(os.path.join("./train/",filename[i]))
    image_resized = transform.resize(image, (160,240,3),anti_aliasing=True)
    train[i] = image_resized


In [114]:

oc = OrdinalEncoder()
labels_np = np.reshape(labels.to_numpy(),(-1,1))
labels_enc = to_categorical(oc.fit_transform(labels_np),num_classes=15)

In [115]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(128, (5, 5), activation = 'relu', input_shape = (160, 240, 3)))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(256, (5, 5), activation = 'relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Conv2D(512, (5, 5), activation = 'relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(256, (3, 3), activation = 'relu'))
model.add(keras.layers.MaxPooling2D((2, 2)))
model.add(keras.layers.Conv2D(2, (3, 3), activation = 'relu'))
model.add(keras.layers.GlobalAveragePooling2D()) 
model.add(keras.layers.Dense(15, activation = 'softmax'))

In [116]:
callback = keras.callbacks.EarlyStopping(restore_best_weights=True,monitor="val_accuracy",patience=10)

In [117]:
model.compile(optimizer = "adam", loss = keras.losses.categorical_crossentropy, metrics = ['accuracy']) #minimizziamo la loss function H(m,b) = -log(pm)pm - log(pb)pb 

In [ ]:
EPOCHS = 20

history = model.fit(train[:5],labels_enc[:5],epochs = EPOCHS, validation_split = 0.1, verbose = True ,callbacks = [callback])

In [ ]:
# grafico di alcuni parametri dell'addestramento della rete

import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label = 'Training accuracy')
plt.plot(epochs, val_acc, 'b', label = 'Validation accuracy')
plt.title('Training and validation accuracy')
plt.figure()
plt.plot(epochs, loss, 'r', label = 'Training Loss')
plt.plot(epochs, val_loss, 'b', label = 'Validation Loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()